In [1]:
import sys
import os
import sklearn
import pandas as pd
import numpy as np
import xml
import re
from pprint import pprint

print(sys.version)

3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]


In [2]:
# Download the datasets if they are not already downloaded
if 'data' not in os.listdir('.'):
    !mkdir './data'
if not os.listdir('./data'):
    !wget https://hotell.difi.no/download/nav/ledige-stillinger/2017?download -O ./data/stillingsannonser.csv
    !wget https://hotell.difi.no/download/nav/stillingstekster/2017?download -O ./data/stillingsannonser-beskrivelser.csv


--2018-07-31 08:56:25--  https://hotell.difi.no/download/nav/ledige-stillinger/2017?download
Resolving hotell.difi.no (hotell.difi.no)... 93.94.10.30
Connecting to hotell.difi.no (hotell.difi.no)|93.94.10.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application~/vnd.ms-excel~]
Saving to: ‘./data/stillingsannonser.csv’

./data/stillingsann     [   <=>              ]  49.07M  9.14MB/s    in 6.4s    

2018-07-31 08:56:31 (7.73 MB/s) - ‘./data/stillingsannonser.csv’ saved [51454162]

--2018-07-31 08:56:32--  https://hotell.difi.no/download/nav/stillingstekster/2017?download
Resolving hotell.difi.no (hotell.difi.no)... 93.94.10.30
Connecting to hotell.difi.no (hotell.difi.no)|93.94.10.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application~/vnd.ms-excel~]
Saving to: ‘./data/stillingsannonser-beskrivelser.csv’

./data/stillingsann     [   <=>              ] 341.82M  9.01MB/s    in 42s     

2018-07-31 

In [3]:
# Load dataset from csv
data_dir_path = os.path.join('.', 'data')
job_ads_filename = 'stillingsannonser.csv'
job_ads_desc_filename = 'stillingsannonser-beskrivelser.csv'

df_ads = pd.read_csv(
    os.path.join(data_dir_path, job_ads_filename),
    sep=';'
)

df_ads_desc = pd.read_csv(
    os.path.join(data_dir_path, job_ads_desc_filename),
    sep=';'
)

# Merge datasets into one
df = pd.merge(df_ads, df_ads_desc)

df.head()

,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,...,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,statistikkperiode,stillingsbeskrivelse
0,101201612000013,101,12.12.2016,15.01.2017,201701,1,Reg av arb.giver på nav.no,01,Østfold,0101,...,Undervisning,2342,Førskolelærere,Barnehagelærer,974122545,YNGLINGEN BARNEHAGE,1,Pedagogisk leder,201701.0,<p>Pedagogisk leder i 73% stilling</p>
1,101201612000015,101,14.12.2016,28.12.2016,201701,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Undervisning,2351,Spesialister i pedagogikk,Studiekonsulent,973967592,HØGSKOLEN I ØSTFOLD,1,Rådgiver - studentopptak og internasjonalisering,201701.0,Jobbnorge ID: 131633<br><br><p># i Østfold har...
2,101201612000016,101,14.12.2016,31.01.2017,201701,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Undervisning,2310,Universitets- og høyskolelektorer/-lærere,Førsteamanuensis,973967592,HØGSKOLEN I ØSTFOLD,1,Undervisnings- og forskerstilling i musikk,201701.0,Jobbnorge ID: 131849<br><br><p>Vi søker en dyk...
3,101201612000017,101,15.12.2016,20.01.2017,201701,1,Reg av arb.giver på nav.no,01,Østfold,0101,...,Reiseliv og transport,9412,Kjøkkenassistenter,Anretningsassistent,973070452,AXXE AS,1,Kantinemedarbeider,201701.0,<p>Vi søker en medarbeider i 80% vikariat til...
4,101201612000018,101,15.12.2016,15.01.2017,201701,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Undervisning,2310,Universitets- og høyskolelektorer/-lærere,Lærer (universitet og høyskole),973967592,HØGSKOLEN I ØSTFOLD,1,Undervisnings- og forskerstilling i kroppsøving,201701.0,Jobbnorge ID: 132043<br><br><p>Vi søker en dyk...


In [16]:
whitespace_pattern = re.compile(r'\s+')

def remove_tags(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def remove_whitespace(text):
    return whitespace_pattern.sub(' ', text)

# Select subset of columns
col = [
    'stilling_kilde', 
    'stillingsnummer', 
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'arbeidssted_fylkesnummer',
    'arbeidssted_kommunenummer',
    'virksomhet_navn',
    'arbeidssted_fylke',
]
df2 = df[col]


# Remove html tags from description
df2[['stillingsbeskrivelse']] = df['stillingsbeskrivelse'].map(remove_tags).map(remove_whitespace)
df2['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df2['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df2['contains_nav'] = df['stillingsbeskrivelse'].map(lambda x: 'nav' in x)

print(df2.shape)
df2.head()




/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

(148144, 11)


/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,contains_nav
0,Reg av arb.giver på nav.no,101201612000013,Pedagogisk leder i 73% stilling,Undervisning,01,0101,YNGLINGEN BARNEHAGE,Østfold,True,False,False
1,Overført fra arbeidsgiver,101201612000015,Jobbnorge ID: 131633# i Østfold har vi ledig e...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD,Østfold,False,False,True
2,Overført fra arbeidsgiver,101201612000016,Jobbnorge ID: 131849Vi søker en dyktig medarbe...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD,Østfold,False,False,True
3,Reg av arb.giver på nav.no,101201612000017,Vi søker en medarbeider i 80% vikariat til den...,Reiseliv og transport,01,0101,AXXE AS,Østfold,True,False,False
4,Overført fra arbeidsgiver,101201612000018,Jobbnorge ID: 132043Vi søker en dyktig medarbe...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD,Østfold,False,False,True


In [36]:
from sklearn.model_selection import train_test_split

data_subset_size = len(df2) # Just use a small dataset when experimenting or it will take too much time
target_column_name = 'yrke_grovgruppe' # Which label to predict
random_seed = 305

data_sample = df2.sample(n=data_subset_size, random_state=random_seed)

# Make training and test sets of the data. 
X_train, X_test, y_train, y_test = train_test_split(
    data_sample, 
    data_sample[target_column_name],
    test_size=.5,
    random_state=random_seed
)

X_train

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,contains_nav
92558,Annonsert i media,1165201702000016,Vil du arbeide på et sykehjem som ønsker å gå ...,"Helse, pleie og omsorg",11,1103,SLÅTTHAUG SYKEHJEM,Rogaland,False,True,True
36056,Overført fra arbeidsgiver,316201709000557,Det er ledig fast stilling som overlege ved L...,"Helse, pleie og omsorg",03,0301,OSLO UNIVERSITETSSYKEHUS HF,Oslo,False,False,True
28478,Overført fra arbeidsgiver,313201710000826,I denne jobben bidrar du virkelig til Starbuck...,Butikk- og salgsarbeid,03,0301,UMOE RESTAURANT GROUP COFFEE AS,Oslo,False,False,False
96341,Overført fra arbeidsgiver,1208201706000087,RAINFALL SØKER EN FORDØMT GODE UTVIKLER!Hvis d...,Ingeniør- og ikt-fag,12,1201,RAINFALL AS,Hordaland,False,False,False
10446,Overført fra arbeidsgiver,219201705000263,Vi søker PT på våre Actic-sentere i Oslo!Actic...,Serviceyrker og annet arbeid,03,0301,ACTIC NORGE AS,Oslo,False,False,False
144855,Annonsert i media,1940201710000003,Kåfjord kommune har følgende ledige stillinger...,"Helse, pleie og omsorg",19,1940,KÅFJORD KOMMUNE OPPVEKST OG OMSORG,Troms,False,True,True
132775,Overført fra arbeidsgiver,1805201708000020,"Vil du være tett på konjunkturen, kjenne på p...",Meglere og konsulenter,18,1805,ADECCO NORGE AS,Nordland,False,False,False
81216,Annonsert i media,1004201704000007,Inntil 100% fast undervisningsstilling i engel...,Undervisning,10,1004,FLEKKEFJORD VIDEREGÅENDE SKOLE,Vest-Agder,False,True,False
36391,Annonsert i media,316201710000261,Deloitte er en av verdens største leverandører...,Meglere og konsulenter,03,0301,DELOITTE AS,Oslo,False,True,False
73798,Overført fra arbeidsgiver,806201709000069,Seksjon for familiebehandling er en av seks s...,Akademiske yrker,08,0806,SYKEHUSET TELEMARK HF,Telemark,False,False,False


In [37]:
# Verify that the data looks as expected
X_test

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,contains_nav
94692,Annonsert i media,1204201703000009,66% faste miljøarbeiderstilling på natt - # bo...,"Helse, pleie og omsorg",12,1201,JOHAN RONGVEDSVEG BOFELLESSKAP,Hordaland,False,True,False
135565,Annonsert i media,1837201702000039,Miljøtjenesten har behov for sommervikarer i p...,"Helse, pleie og omsorg",18,1837,MELØY KOMMUNE MOSVOLDTUNET HUS 2A,Nordland,False,True,False
65607,Fra Eures,620201707000004,Vi søker: Bartender i Barocken bar. Søkeren må...,Reiseliv og transport,06,0620,BARDØLA HØYFJELLSHOTELL AS,Buskerud,False,False,False
117060,Reg av arb.giver på nav.no,1547201706000004,# har ledig fra 60-100% stilling som pedagogis...,Undervisning,15,1547,TROLLMYRA BARNEHAGE AUKRA AS,Møre og Romsdal,True,False,False
136884,Reg av arb.giver på nav.no,1860201707000004,Vi søker dyktig barne og ungdomsarbeider til v...,Barne- og ungdomsarbeid,18,1860,TROLLKNAUSEN BARNEHAGE AS,Nordland,True,False,False
5710,Reg av arb.giver på nav.no,136201703000014,Vi søker hjelpemann til vår flyttebil med erf...,Industriarbeid,01,0136,MULTIDRIFT ØST AS,Østfold,True,False,False
95362,Annonsert i media,1206201703000016,Forsvarsbygg søkjer etter motiverte sommarvika...,Serviceyrker og annet arbeid,12,1201,FORSVARSBYGG,Hordaland,False,True,False
102492,Annonsert i media,1210201707000034,KIWI minipris er en av landets mest ekspansive...,Butikk- og salgsarbeid,12,1201,NG KIWI VEST AS,Hordaland,False,True,False
56417,Reg av arb.giver på nav.no,429201706000012,Vi har ledig vikariat i 80 % stilling som bibl...,Akademiske yrker,04,0429,ÅMOT KOMMUNE SENTRALADMINISTRASJON,Hedmark,True,False,False
71369,Annonsert i media,710201711000054,Statkrafts fjernvarmevirksomhet har drevet med...,Serviceyrker og annet arbeid,07,0710,STATKRAFT AS,Vestfold,False,True,False


In [38]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import category_encoders as ce


# Used to select one column from a pandas DataFrame
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x[self.column]

# Convert a pandas DataFrame to an np array
class DataFrameToArrayTransformer(BaseEstimator, TransformerMixin):    
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return X.values
    

# Simply count the number of words in a string
class WordCountVectorizer(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        #return np.array(X.map(lambda x: len(x.split(' '))), dtype=pd.Series)
        #return np.transpose(X.map(lambda x: len(x.split(' '))).as_matrix())
        return X.map(lambda x: len(x.split(' '))).values.reshape(-1, 1)

# Check if a string contains a word
class ContainsWordVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, word):
        self.word = word
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        result = np.array([self.word in x for x in X]).reshape(-1, 1)
        return result


# This is the pipeline that our data will flow through for training and testing
pipeline = Pipeline([
    # Use FeatureUnion to combine the features from different columns
    ('union', FeatureUnion(
        transformer_list=[
            ('text', Pipeline([
                ('selector', ItemSelector('stillingsbeskrivelse')),
                ('counts', CountVectorizer(max_df=.2)),
                ('tf_idf', TfidfTransformer())
            ])),
            ('company', Pipeline([
                ('selector', ItemSelector('virksomhet_navn')),
                ('counts', CountVectorizer()),
            ])),
            ('source', Pipeline([
                ('selector', ItemSelector('stilling_kilde')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ce.BinaryEncoder())
            ])),
            ('kommune', Pipeline([
                ('selector', ItemSelector('arbeidssted_kommunenummer')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ce.BinaryEncoder())
            ])),
            ('fylke', Pipeline([
                ('selector', ItemSelector('arbeidssted_fylkesnummer')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ce.BinaryEncoder())
            ])),
            ('from_nav', Pipeline([
                ('selector', ItemSelector('stillingsbeskrivelse')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ContainsWordVectorizer('nav'))
            ])),
            ('word_count', Pipeline([
                ('selector', ItemSelector('stillingsbeskrivelse')),
                ('count', WordCountVectorizer()),
            ])),
        ],
        transformer_weights = {
            'text': 1.0,
            'company': .0,
            'source': .0,
            'fylke': .0,
            'kommune': .0,
            'from_nav': .0,
            'word_count': .0,
        },
    )),

    # Use a SVC classifier on the combined features
    ('svc', LinearSVC()),
])

# Train the ML algorithm with the training data
pipeline = pipeline.fit(X_train, y_train)

In [39]:
# Test the data by running the test data set on the trained pipeline
predicted = pipeline.predict(X_test)
pprint(predicted)
np.mean(predicted == y_test)

array(['Helse, pleie og omsorg', 'Helse, pleie og omsorg',
       'Reiseliv og transport', ..., 'Helse, pleie og omsorg',
       'Undervisning', 'Ingeniør- og ikt-fag'], dtype=object)


0.8204314720812182

In [40]:
from sklearn import metrics

# Print some stats of our results
print(metrics.classification_report(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))

                                    precision    recall  f1-score   support

                  Akademiske yrker       0.73      0.73      0.73      5080
           Barne- og ungdomsarbeid       0.82      0.73      0.77      2392
            Butikk- og salgsarbeid       0.84      0.87      0.86      5889
                    Bygg og anlegg       0.84      0.85      0.85      4883
            Helse, pleie og omsorg       0.90      0.95      0.93     18744
                    Industriarbeid       0.75      0.71      0.73      3052
Ingen yrkesbakgrunn eller uoppgitt       0.49      0.17      0.26       197
              Ingeniør- og ikt-fag       0.76      0.81      0.78      7164
       Jordbruk, skogbruk og fiske       0.77      0.60      0.67       450
                      Kontorarbeid       0.69      0.63      0.66      3018
                            Ledere       0.57      0.49      0.53      1827
            Meglere og konsulenter       0.66      0.62      0.64      3169
           

In [41]:
# Try the pipeline with some custom data

test_dataframe = df2[0:1]
test_dataframe[0:1]['stillingsbeskrivelse'] = 'ingeniør'
test_dataframe[0:1]['virksomhet_navn'] = 'Statoil'
prediction = pipeline.predict(test_dataframe)
pprint(prediction)

array(['Ingeniør- og ikt-fag'], dtype=object)


In [11]:
test_dataframe

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,is_from_nav,from_media,contains_nav
0,Reg av arb.giver på nav.no,101201612000013,are you interested in sales management key acc...,Undervisning,01,0101,Statoil,True,False,False
